<script type="text/javascript"
  src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.0/MathJax.js?config=TeX-AMS_CHTML">
</script>
<script type="text/x-mathjax-config">
  MathJax.Hub.Config({
    tex2jax: {
      inlineMath: [['$','$'], ['\\(','\\)']],
      processEscapes: true},
      jax: ["input/TeX","input/MathML","input/AsciiMath","output/CommonHTML"],
      extensions: ["tex2jax.js","mml2jax.js","asciimath2jax.js","MathMenu.js","MathZoom.js","AssistiveMML.js", "[Contrib]/a11y/accessibility-menu.js"],
      TeX: {
      extensions: ["AMSmath.js","AMSsymbols.js","noErrors.js","noUndefined.js"],
      equationNumbers: {
      autoNumber: "AMS"
      }
    }
  });
</script>

Jakub Musiał

# **AOD - lab2**

## **Exercise 2 - Best route**

Knowing a graph of connections between cities, travel costs and times, determine the cheapest route the travel time of which does not exceed $T$

In [1]:
using JuMP
using GLPK

import JSON

#### **Data and utils**

In [2]:
# Data initialization
data_general = JSON.parse(read("data.json", String))
# Extraxt exercise 2 data from the general dictionary
data = data_general["ex2"]

Dict{String, Any} with 4 entries:
  "destination" => "Kyiv"
  "source"      => "Madrid"
  "T"           => 40
  "connections" => Dict{String, Any}("Warsaw"=>Dict{String, Any}("Kyiv"=>Dict{S…

In [3]:
cities = sort(collect(keys(data["connections"])))
src = data["source"]
dest = data["destination"]
T = data["T"]
cities, src, dest, T

(["Amsterdam", "Berlin", "Bern", "Bratislava", "Brussels", "Kyiv", "Madrid", "Paris", "Prague", "Rome", "Vienna", "Warsaw"], "Madrid", "Kyiv", 40)

In [4]:
neighbors(city::String) = keys(data["connections"][city])
time(ci::String, cj::String) = data["connections"][ci][cj]["time"]
cost(ci::String, cj::String) = data["connections"][ci][cj]["cost"]

cost (generic function with 1 method)

#### **Build the model**

Notation:
* $C = \text{cities}$

In [5]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

**Predictor variables:**

$x_{c_i, c_j}$ where $(c_i, c_j) \text{ } \epsilon \text{ } C^2$ - indicator that the best route includes the path from city $c_i$ to city $c_j$

In [6]:
@variable(model, x[cities, cities], Bin)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["Amsterdam", "Berlin", "Bern", "Bratislava", "Brussels", "Kyiv", "Madrid", "Paris", "Prague", "Rome", "Vienna", "Warsaw"]
    Dimension 2, ["Amsterdam", "Berlin", "Bern", "Bratislava", "Brussels", "Kyiv", "Madrid", "Paris", "Prague", "Rome", "Vienna", "Warsaw"]
And data, a 12×12 Matrix{VariableRef}:
 x[Amsterdam,Amsterdam]   x[Amsterdam,Berlin]   …  x[Amsterdam,Warsaw]
 x[Berlin,Amsterdam]      x[Berlin,Berlin]         x[Berlin,Warsaw]
 x[Bern,Amsterdam]        x[Bern,Berlin]           x[Bern,Warsaw]
 x[Bratislava,Amsterdam]  x[Bratislava,Berlin]     x[Bratislava,Warsaw]
 x[Brussels,Amsterdam]    x[Brussels,Berlin]       x[Brussels,Warsaw]
 x[Kyiv,Amsterdam]        x[Kyiv,Berlin]        …  x[Kyiv,Warsaw]
 x[Madrid,Amsterdam]      x[Madrid,Berlin]         x[Madrid,Warsaw]
 x[Paris,Amsterdam]       x[Paris,Berlin]          x[Paris,Warsaw]
 x[Prague,Amsterdam]      x[Prague,Berlin]         x[Prague,Warsaw]


**Constraints:** 

* Indicators must be binary
  
  $(\forall{(c, c') \text{ } \epsilon \text{ } C^2})(x_{c, c'} \text{ } \epsilon \text{ } \{0, 1\})$

* If a city $c'$ is not a neighbor of $c$, the indicator $x_{c, c'}$ must be equal to 0
  
  $(\forall{c \epsilon C})(\forall{c' \epsilon C \setminus \text{neighbors}(c)})(x_{c, c'} = 0)$

* The source must be exited
  
  $
  \left\lbrace 
  \begin{array}{l}
  (\sum_{c \text{ } \epsilon \text{ neighbors}(src)} x_{src, c} = 1) \newline
  (\sum_{c \text{ } \epsilon \text{ neighbors}(src)} x_{c, src} = 0)
  \end{array}\right.
  $

* The destination cannot be exited

  $
  \left\lbrace 
  \begin{array}{l}
  (\sum_{c \text{ } \epsilon \text{ neighbors}(dest)} x_{dest, c} = 0) \newline
  (\sum_{c \text{ } \epsilon \text{ neighbors}(dest)} x_{c, dest} = 1)
  \end{array}\right.
  $

* All cities other than the source and destination must be exited the same ammount of times as the are entered
  
  $(\forall{c \epsilon C \setminus \{src, dest\}})(\sum_{c' \epsilon C} x_{c, c'} = \sum_{c' \epsilon C} x_{c', c})$

* The total trip time cannot exceed given time $T$
  
  $\sum_{c \epsilon C} \sum_{c' \epsilon \text{ neighbors}(c)} x_{c, c'} * time(c, c') \leq T$

In [7]:
for ci in cities, cj in cities 
    if !(cj in neighbors(ci))
        @constraint(model, x[ci, cj] == 0)
    end
end

@constraints(model, begin
    sum(x[src, :]) == 1
    sum(x[:, src]) == 0
    sum(x[dest, :]) == 0
    sum(x[:, dest]) == 1
    sum(x[ci, cj] * time(ci, cj) for ci in cities, cj in neighbors(ci)) <= T
end)

for c in cities
    if (c != src && c != dest)
        @constraint(model, sum(x[c, :]) == sum(x[:, c]))
    end
end

**Objective:** minimal (source -> destination) trip cost

$min(\sum_{(c_i \epsilon C)} \sum_{c_j \epsilon \text{neigbors}(c_i)} x_{c_i, c_j} * cost(c_i, c_j))$

In [8]:
@objective(model, Min, sum(x[ci, cj] * cost(ci, cj) for ci in cities, cj in neighbors(ci)))

x[Amsterdam,Brussels] + 9 x[Amsterdam,Berlin] + 13 x[Berlin,Warsaw] + 5 x[Berlin,Brussels] + x[Berlin,Prague] + 17 x[Berlin,Bern] + 13 x[Berlin,Amsterdam] + 17 x[Berlin,Vienna] + 14 x[Bern,Rome] + 10 x[Bern,Paris] + 10 x[Bern,Berlin] + 24 x[Bern,Vienna] + 6 x[Bratislava,Kyiv] + 8 x[Bratislava,Warsaw] + 7 x[Bratislava,Prague] + x[Bratislava,Vienna] + 4 x[Brussels,Paris] + 21 x[Brussels,Berlin] + x[Brussels,Amsterdam] + 8 x[Kyiv,Warsaw] + 32 x[Kyiv,Bratislava] + 20 x[Madrid,Paris] + 20 x[Paris,Madrid] + 4 x[Paris,Rome] + 11 x[Paris,Brussels] + 2 x[Paris,Bern] + x[Prague,Warsaw] + 8 x[Prague,Bratislava] + 7 x[Prague,Berlin] + 7 x[Prague,Vienna] + 16 x[Rome,Paris] + 19 x[Rome,Bern] + 28 x[Rome,Vienna] + 29 x[Vienna,Rome] + x[Vienna,Bratislava] + 4 x[Vienna,Prague] + 18 x[Vienna,Bern] + 2 x[Vienna,Berlin] + x[Warsaw,Kyiv] + 13 x[Warsaw,Bratislava] + 2 x[Warsaw,Prague] + 11 x[Warsaw,Berlin]

In [9]:
# Optimize the model
optimize!(model)
solution_summary(model)

* Solver : GLPK

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Solution is optimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 5.50000e+01
  Objective bound    : 5.50000e+01
  Relative gap       : 0.00000e+00

* Work counters
  Solve time (sec)   : 3.00002e-03


#### **Results**

* Best route

In [11]:
city = src
total_cost = 0
total_time = 0

while city != dest 
    for neighbor in neighbors(city)
        if value(x[city, neighbor]) == 1
            println(city, " -> ", neighbor)
            total_cost += cost(city, neighbor)
            total_time += time(city, neighbor)
            city = neighbor
        end
    end
end

println("\ntotal cost: ", total_cost)
println("total time: ", total_time, " <= T: ", total_time <= T)

Madrid -> Paris
Paris -> Brussels
Brussels -> Amsterdam
Amsterdam -> Berlin
Berlin -> Warsaw
Warsaw -> Kyiv

total cost: 55
total time: 40 <= T: true


* Best route with non-integer predictor variables

In [12]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

# Predictor variables
@variable(model, x[cities, cities] >= 0) # NON-INTEGER VALUES

# Constraints
for ci in cities, cj in cities 
    if !(cj in neighbors(ci))
        @constraint(model, x[ci, cj] == 0)
    end
end
@constraints(model, begin
    sum(x[src, :]) == 1
    sum(x[:, src]) == 0
    sum(x[dest, :]) == 0
    sum(x[:, dest]) == 1
    sum(x[ci, cj] * time(ci, cj) for ci in cities, cj in neighbors(ci)) <= T
end)
for c in cities
    if (c != src && c != dest)
        @constraint(model, sum(x[c, :]) == sum(x[:, c]))
    end
end

# objective
@objective(model, Min, sum(x[ci, cj] * cost(ci, cj) for ci in cities, cj in neighbors(ci)))

# Optimize the model
optimize!(model)

# Results
city = src
total_cost = 0
total_time = 0

while city != dest 
    for neighbor in neighbors(city)
        if value(x[city, neighbor]) == 1
            println(city, " -> ", neighbor)
            total_cost += cost(city, neighbor)
            total_time += time(city, neighbor)
            city = neighbor
        end
    end
end

println("\ntotal cost: ", total_cost)
println("total time: ", total_time, " <= T: ", total_time <= T)

Madrid -> Paris
Paris -> Brussels
Brussels -> Amsterdam
Amsterdam -> Berlin
Berlin -> Warsaw
Warsaw -> Kyiv

total cost: 55
total time: 40 <= T: true


* Best route with non-integer predictor variables and no maximum time constraint

In [13]:
model = Model()
set_optimizer(model, GLPK.Optimizer)

# Predictor variables
@variable(model, x[cities, cities] >= 0) # NON-INTEGER VALUES

# Constraints
for ci in cities, cj in cities 
    if !(cj in neighbors(ci))
        @constraint(model, x[ci, cj] == 0)
    end
end
@constraints(model, begin
    sum(x[src, :]) == 1
    sum(x[:, src]) == 0
    sum(x[dest, :]) == 0
    sum(x[:, dest]) == 1
    # sum(x[ci, cj] * time(ci, cj) for ci in cities, cj in neighbors(ci)) <= T
end)
for c in cities
    if (c != src && c != dest)
        @constraint(model, sum(x[c, :]) == sum(x[:, c]))
    end
end

# objective
@objective(model, Min, sum(x[ci, cj] * cost(ci, cj) for ci in cities, cj in neighbors(ci)))

# Optimize the model
optimize!(model)

# Results
city = src
total_cost = 0
total_time = 0

while city != dest 
    for neighbor in neighbors(city)
        if value(x[city, neighbor]) == 1
            println(city, " -> ", neighbor)
            total_cost += cost(city, neighbor)
            total_time += time(city, neighbor)
            city = neighbor
        end
    end
end

println("\ntotal cost: ", total_cost)
println("total time: ", total_time, " <= T: ", total_time <= T)

Madrid -> Paris
Paris -> Bern
Bern -> Berlin
Berlin -> Prague
Prague -> Warsaw
Warsaw -> Kyiv

total cost: 35
total time: 48 <= T: false
